In [1]:
import pandas as pd
import spacy
from collections import Counter

In [6]:
data = pd.read_csv("20 newsgroups/20newsgroups.csv", on_bad_lines='skip', sep=';')
data.dropna()
data = data.iloc[:,1:3]


nlp = spacy.load("en_core_web_sm")
nlp.Defaults.stop_words.add("  ")
nlp.Defaults.stop_words.add("   ")
nlp.Defaults.stop_words.add(" ")
nlp.Defaults.stop_words.add(">")

group = 0 --> "atheism"

In [7]:
data0 = data.drop(data[data["group"] != 0].index)


In [8]:
word_freq0 = Counter()
for text in data0["text"]:
    text = str(text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct)]
    word_freq0.update(filtered_words)
print(word_freq0.most_common(20))

[('god', 575), ('people', 476), ("don\\'t", 433), ('think', 357), ('believe', 279), ('know', 275), ('like', 272), ('       ', 267), ('=', 251), ('religion', 248), ('    ', 245), ('it\\', 235), ('way', 221), ('atheism', 220), ('time', 220), ('said', 216), ('           ', 215), ('atheists', 212), ('jesus', 210), ('evidence', 208)]


group = 1 --> "graphics"

In [10]:
data1 = data.drop(data[data["group"] != 1].index)

In [13]:
word_freq1 = Counter()
for text in data1["text"]:
    text = str(text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct)]
    word_freq1.update(filtered_words)
print(word_freq1.most_common(20))

[('=', 7167), ('image', 1113), ('    ', 809), ('jpeg', 756), ('graphics', 723), ('file', 602), ('     ', 550), ('images', 505), ('available', 451), ('data', 449), ('        ', 444), ('software', 439), ('use', 439), ('files', 427), ('format', 425), ('+', 412), ('      ', 408), ('<', 399), ('color', 391), ('program', 385)]


group = 2 --> space

In [14]:
data2 = data.drop(data[data["group"] != 2].index)

In [17]:
word_freq2 = Counter()
for text in data2["text"]:
    text = str(text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct)]
    word_freq2.update(filtered_words)
print(word_freq2.most_common(20))

[('=', 2060), ('    ', 1967), ('space', 1185), ('     ', 917), ('\\t', 547), ('           ', 504), ('      ', 462), ('$', 405), ('like', 378), ('launch', 375), ('nasa', 346), ('earth', 317), ('time', 315), ('system', 303), ('orbit', 300), ('1', 282), ('shuttle', 281), ('data', 271), ("don\\'t", 254), ('mission', 251)]


group = 3 --> religion.misc

In [16]:
data3 = data.drop(data[data["group"] != 3].index)

In [18]:
word_freq3 = Counter()
for text in data3["text"]:
    text = str(text)
    doc = nlp(text)
    filtered_words = [token.text.lower() for token in doc if (not token.is_stop and not token.is_punct)]
    word_freq3.update(filtered_words)
print(word_freq3.most_common(20))

[('god', 582), ('           ', 513), ('people', 393), ('jesus', 374), ('    ', 268), ("don\\'t", 262), ('bible', 256), ('know', 247), ('     ', 241), ('think', 239), ('=', 228), ('christian', 215), ('like', 212), ('jehovah', 191), ('believe', 188), ('good', 187), ('way', 186), ('lord', 185), ('said', 183), ('christ', 183)]


Filter words --> keine Adjektive und Verben

Ergebnis: Noun, propn, num --> schlechter prec: 0,596 --> Information verloren
Ergebnis: Noun, propn, num, adv --> schlechter prec: 0,57 --> negative Effekt

In [39]:
def filter_words(word_list):
    # Wandle die Wörter in spaCy-Dokumente um und prüfe die POS-Tags
    filtered_words = [word for word in word_list if nlp(word)[0].pos_ in ["NOUN", "PROPN", "NUM", "ADV"]]
    return filtered_words

filterd0 = filter_words(word_freq0)
filterd1 = filter_words(word_freq1)
filterd2 = filter_words(word_freq2)
filterd3 = filter_words(word_freq3)


In [36]:
print(filterd0[:50])

['request', 'charley', 'wingate', 'mail', 'mr.', 'post', 'tangential', 'articles', 'postings', 'dozen', 'posts', 'challenge', 'questions', 'level', 'subject', 'tactic', 'desire', 'answers', 'manner', '1', 'ignore', 'future', 'address', 'announces', '2', 'answer', 'honesty', 'christian', 'question', 'deaths', 'atheists', 'theists', 'list', 'atheist', 'genocide', 'bit', 'history', 'somone', 'wars', 'hitler', 'example', 'statement', 'peace', 'advice', 'beliefs', 'nanci', 'size', 'armies', 'duration', 'numbers']


In [48]:
filterd = filterd0[:500]+filterd1[:500]+filterd2[:500]+filterd3[:500]+["groupID"]
filterdData = pd.DataFrame(columns=filterd)
filterdData.to_csv("20newsgroups_filterd.csv", index=False)
filterdData.head()

,request,charley,wingate,mail,mr.,post,tangential,articles,postings,dozen,...,abraham,moses,official,egypt,elijah,priest,david,mere,shepherd,groupID


In [41]:
rows = []

# Iteriere über die Zeilen im DataFrame
for index, row in data.iterrows():
    text = str(row['text'])
    group = str(row['group'])

    doc = nlp(text)
    # Zähle die Häufigkeiten der Wörter aus der Liste
    count_in_top_1000 = Counter(token.text.lower() for token in doc if token.is_alpha)
    # Häufigkeiten in einem Dictionary sammeln
    haeufigkeiten = {word: count_in_top_1000.get(word, 0) for word in filterd}
    haeufigkeiten['groupID'] = group
    # Füge das Dictionary der Liste hinzu
    rows.append(haeufigkeiten)

# Erstelle das endgültige DataFrame aus der Liste von Dictionaries
filterdData = pd.DataFrame(rows)

# Speichere das Ergebnis in einer CSV-Datei
filterdData.to_csv("20newsgroups_filterd.csv", index=False)
print(filterdData.head())

   request  charley  wingate  mail  mr.  post  tangential  articles  postings  \
0        0        0        0     0    0     0           0         0         0   
1        0        0        0     0    0     0           0         0         0   
2        0        0        0     0    0     0           0         0         0   
3        1        3        5     1    0     1           1         3         1   
4        0        0        0     0    0     0           0         0         0   

   dozen  ...  abraham  moses  official  egypt  elijah  priest  david  mere  \
0      0  ...        0      0         0      0       0       0      0     0   
1      0  ...        0      0         0      0       0       0      0     0   
2      0  ...        0      0         0      0       0       0      0     0   
3      1  ...        0      0         0      0       0       0      0     0   
4      0  ...        0      0         0      0       0       0      0     0   

   shepherd  groupID  
0         0    

Neuer Ansatz: Entfernung der Wörter die in den anderen Listen zu häufig sind --> Eindeutigkeit verbessern 

In [42]:
# Alle Kategorien zusammen in ein Dictionary packen
all_words = {
    "Atheism": word_freq0,
    "Graphics": word_freq1,
    "Space": word_freq2,
    "Religion": word_freq3
}

# Dictionary für die gemeinsamen Wörter
common_words = {category: {} for category in all_words}

# Durchlaufe alle Wörter und prüfe, wie oft sie in anderen Kategorien vorkommen
for category, words in all_words.items():
    for word in words:
        for other_category, other_words in all_words.items():
            if category != other_category:
                if word in other_words:
                    common_words[category][word] = common_words[category].get(word, 0) + 1

# Anzeigen, welche Wörter in mehreren Kategorien erscheinen
for category, words in common_words.items():
    print(f"Gemeinsame Wörter in der Kategorie {category}:")
    for word, count in words.items():
        print(f"{word}: {count} weitere Kategorie(n)")


Gemeinsame Wörter in der Kategorie Atheism:
request: 3 weitere Kategorie(n)
like: 3 weitere Kategorie(n)
respond: 3 weitere Kategorie(n)
challenges: 2 weitere Kategorie(n)
judging: 2 weitere Kategorie(n)
e: 3 weitere Kategorie(n)
mail: 3 weitere Kategorie(n)
appear: 3 weitere Kategorie(n)
clear: 3 weitere Kategorie(n)
mr.: 3 weitere Kategorie(n)
intends: 2 weitere Kategorie(n)
continue: 3 weitere Kategorie(n)
post: 3 weitere Kategorie(n)
unrelated: 2 weitere Kategorie(n)
articles: 3 weitere Kategorie(n)
ingoring: 1 weitere Kategorie(n)
postings: 3 weitere Kategorie(n)
noted: 3 weitere Kategorie(n)
dozen: 3 weitere Kategorie(n)
posts: 3 weitere Kategorie(n)
answered: 3 weitere Kategorie(n)
single: 3 weitere Kategorie(n)
challenge: 3 weitere Kategorie(n)
hopes: 3 weitere Kategorie(n)
questions: 3 weitere Kategorie(n)
away: 3 weitere Kategorie(n)
level: 3 weitere Kategorie(n)
best: 3 weitere Kategorie(n)
change: 3 weitere Kategorie(n)
subject: 3 weitere Kategorie(n)
given: 3 weitere Kateg

In [51]:
# Schwelle: Wörter, die in mehr als 2 Kategorien vorkommen, sollen entfernt werden
threshold = 2
words_per_category = 500  # Anzahl der Wörter, die pro Kategorie übrig bleiben sollen

# Alle Kategorien zusammen in ein Dictionary packen
all_words = {
    "Atheism": word_freq0,
    "Graphics": word_freq1,
    "Space": word_freq2,
    "Religion": word_freq3
}

# Dictionary für die gemeinsamen Wörter
common_words = {category: {} for category in all_words}

# Durchlaufe alle Wörter und prüfe, wie oft sie in anderen Kategorien vorkommen
for category, words in all_words.items():
    for word in words:
        for other_category, other_words in all_words.items():
            if category != other_category and word in other_words:
                common_words[category][word] = common_words[category].get(word, 0) + 1

# Filtere Wörter, die in mehr als 2 Kategorien vorkommen
filtered_common_words = {category: {} for category in all_words}
final_words = []  # Liste der übrig gebliebenen Wörter nach Filterung

# Durchlaufe alle Kategorien und filtere Wörter, die in mehr als 2 Kategorien vorkommen
for category, words in common_words.items():
    for word, count in words.items():
        if count <= threshold:  # Entferne Wörter, die in mehr als 2 Kategorien vorkommen
            filtered_common_words[category][word] = count

# Sortiere die Wörter innerhalb jeder Kategorie nach Häufigkeit
for category, words in filtered_common_words.items():
    # Sortiere nach Häufigkeit (absteigend) und wähle die Top 500 aus
    sorted_words = sorted(words.items(), key=lambda x: x[1], reverse=True)
    selected_words = [word for word, _ in sorted_words[:min(words_per_category, len(words))]]
    final_words.extend(selected_words)  # Alle ausgewählten Wörter zur finalen Liste hinzufügen

    print(f"Für Kategorie {category} wurden {len(selected_words)} Wörter ausgewählt:")
    for word in selected_words:
        print(word)

# Zeige die Gesamtzahl der Wörter nach der Auswahl
print("\nGesamtzahl der ausgewählten Wörter:")
print(len(final_words))


Für Kategorie Atheism wurden 500 Wörter ausgewählt:
challenges
judging
intends
unrelated
impress
desire
announces
refuses
atheist
absurd
peace
duration
percentage
troops
civilian
korea
comparisons
nbc
abc
missile
blew
bomb
british
died
jews
poles
camps
justified
rulers
selfish
evident
drastically
corner
stopped
loving
stopping
hadn\'t
lessons
trial
rodney
peers
officers
allegedly
motivations
revision
kidding
silence
excessive
assault
won
intent
harm
akin
seeds
fan
vein
moral
contradictory
pride
sank
manhattan
morally
significance
killing
morality
sentence
\tthere
\tjust
stock
religion
me-
liar
notion
corp
jose
semantics
horribly
guaranteed
happiness
inalienable
proving
classic
serial
killer
admittedly
people\
declaration
independence
infant
golden
survival
implemented
objective
ot
misunderstand
fellow
sophisticated
brains
australian
thrust
innocent
established
spite
distinction
contrary
hi
bible
century
a.d.
4th
325
leaders
dating
manuscripts
probable
divide
deity
behaviour
tendency
he

In [53]:
final_words = final_words+["groupID"]
filterdData = pd.DataFrame(columns=final_words)
filterdData.to_csv("20newsgroups_filterdAdvanced.csv", index=False)
filterdData.head()

,challenges,judging,intends,unrelated,impress,desire,announces,refuses,atheist,absurd,...,questioning,bull,reserved,animal,preferably,stations,broadcast,gross,groupID,groupID


In [54]:
rows = []

# Iteriere über die Zeilen im DataFrame
for index, row in data.iterrows():
    text = str(row['text'])
    group = str(row['group'])

    doc = nlp(text)
    # Zähle die Häufigkeiten der Wörter aus der Liste
    count_in_top_1000 = Counter(token.text.lower() for token in doc if token.is_alpha)
    # Häufigkeiten in einem Dictionary sammeln
    haeufigkeiten = {word: count_in_top_1000.get(word, 0) for word in filterd}
    haeufigkeiten['groupID'] = group
    # Füge das Dictionary der Liste hinzu
    rows.append(haeufigkeiten)

# Erstelle das endgültige DataFrame aus der Liste von Dictionaries
filterdData = pd.DataFrame(rows)

# Speichere das Ergebnis in einer CSV-Datei
filterdData.to_csv("20newsgroupsAdvanced.csv", index=False)
print(filterdData.head())

   request  charley  wingate  mail  mr.  post  tangential  articles  postings  \
0        0        0        0     0    0     0           0         0         0   
1        0        0        0     0    0     0           0         0         0   
2        0        0        0     0    0     0           0         0         0   
3        1        3        5     1    0     1           1         3         1   
4        0        0        0     0    0     0           0         0         0   

   dozen  ...  abraham  moses  official  egypt  elijah  priest  david  mere  \
0      0  ...        0      0         0      0       0       0      0     0   
1      0  ...        0      0         0      0       0       0      0     0   
2      0  ...        0      0         0      0       0       0      0     0   
3      1  ...        0      0         0      0       0       0      0     0   
4      0  ...        0      0         0      0       0       0      0     0   

   shepherd  groupID  
0         0    